In [136]:
using PlotlyJS
include("./utils.jl")
include("./unit_commitment.jl")
include("./plotting.jl")
include("./economic_dispatch.jl")

solve_economic_dispatch (generic function with 1 method)

In [137]:
gen_info, fuels, loads, gen_variable_info, storage_info = read_data()
gen_df = pre_process_generators_data(gen_info, fuels)
gen_variable = pre_process_gen_variable(gen_df, gen_variable_info)
storage_df = pre_process_storage_data(storage_info)
random_loads_df = read_random_demand()

# A spring day
n=100
T_period = (n*24+1):((n+1)*24)

# Filtering data with timeseries according to T_period
gen_variable_multi = gen_variable[in.(gen_variable.hour,Ref(T_period)),:];
loads_multi = loads[in.(loads.hour,Ref(T_period)),:]
random_loads_multi =  random_loads_df[in.(random_loads_df.hour,Ref(T_period)),:];

required_reserve = DataFrame(
    hour = loads_multi[!,:hour],
    reserve_up_MW = 300 .+ loads_multi[!,:demand].*0.05,
    reserve_down_MW = loads_multi[!,:demand].*0.05)

required_energy_reserve = [(row_1.hour, row_2.hour, row_1.reserve_up_MW*(row_1.hour == row_2.hour), row_1.reserve_down_MW*(row_1.hour == row_2.hour)) for row_1 in eachrow(required_reserve), row_2 in eachrow(required_reserve) if row_1.hour <= row_2.hour]
required_energy_reserve = DataFrame(required_energy_reserve)
required_energy_reserve = rename(required_energy_reserve, :1 => :i_hour, :2 => :t_hour, :3 => :reserve_up_MW, :4 => :reserve_down_MW,)

required_energy_reserve_cumulated = [(row_1.hour, row_2.hour, sum(required_reserve[(required_reserve.hour .>= row_1.hour).&(required_reserve.hour .<= row_2.hour),:reserve_up_MW]), sum(required_reserve[(required_reserve.hour .>= row_1.hour).&(required_reserve.hour .<= row_2.hour),:reserve_down_MW])) for row_1 in eachrow(required_reserve), row_2 in eachrow(required_reserve) if row_1.hour <= row_2.hour]
required_energy_reserve_cumulated = DataFrame(required_energy_reserve_cumulated)
required_energy_reserve_cumulated = rename(required_energy_reserve_cumulated, :1 => :i_hour, :2 => :t_hour, :3 => :reserve_up_MW, :4 => :reserve_down_MW,)
;

In [138]:
config = (
    ramp_constraints = true,
    storage = storage_df,
    reserve = required_reserve,
    # energy_reserve = required_energy_reserve,
    enriched_solution = true,
    storage_envelopes = true
)
solution  = solve_economic_dispatch(
        gen_df,
        random_loads_multi,
        gen_variable_multi,
        0.0001;
        config...
        )

Constructing UC...
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2475843
Academic license 2475843 - for non-commercial use only - registered to pa___@imperial.ac.uk
Adding storage...
Adding ramp constraints...
Adding reserve constraints...
Adding storage envelopes...
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: 11th Gen Intel(R) Core(TM) i7-11370H @ 3.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2475843 - for non-commercial use only - registered to pa___@imperial.ac.uk
Optimize a model with 10812 rows, 5766 columns and 27276 nonzeros
Model fingerprint: 0xbc17521b
Variable types: 3942 continuous, 1824 integer (1824 binary)
Coefficient statistics:
  Matrix range     [9e-01, 1e+03]
  Objective range  [1e-01, 5e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 6764 rows an

Excessive output truncated after 525158 bytes.

(generation = 648000×7 DataFrame
    Row │ r_id   hour    production_MW  curtailment_MW  resource               ⋯
        │ Int64  Int64?  Float64?       Float64?        String?                ⋯
────────┼───────────────────────────────────────────────────────────────────────
      1 │    31    2401      163.309       0.0          onshore_wind_turbine   ⋯
      2 │    32    2401        3.18493     0.0          small_hydroelectric
      3 │    33    2401        0.0         0.0          solar_photovoltaic
      4 │    31    2402      169.793       0.0          onshore_wind_turbine
      5 │    32    2402        3.18493     0.0          small_hydroelectric    ⋯
      6 │    33    2402        0.0         0.0          solar_photovoltaic
      7 │    31    2403      115.11        0.0          onshore_wind_turbine
      8 │    32    2403        3.18493     0.0          small_hydroelectric
   ⋮    │   ⋮      ⋮           ⋮              ⋮                         ⋮      ⋱
 647994 │    24    2424  

In [139]:
gdf_demand = groupby(solution.demand, [:demand])
combine(gdf_demand,
[:LOL_MW, :demand_MW] =>
((x, y) -> 
    (LOL_hours=count(x.>0*y.>0),
    LOLP=count(x.>0*y.>0)/count(y.>0),
    LOL_MWh = sum(x),
    LOL_percentage = sum(x)/sum(y),
    Demand_MWh = sum(y),
    )
) => AsTable)


Row,demand,LOL_hours,LOLP,LOL_MWh,LOL_percentage,Demand_MWh
,Symbol,Int64,Float64,Float64,Float64,Float64
1,demand_831,0,0.0,0.0,0.0,55429.9
2,demand_449,0,0.0,0.0,0.0,55392.7
3,demand_48,0,0.0,0.0,0.0,55448.1
4,demand_16,0,0.0,0.0,0.0,55454.4
5,demand_165,0,0.0,0.0,0.0,55451.4
6,demand_690,0,0.0,0.0,0.0,55389.5
7,demand_940,0,0.0,0.0,0.0,55445.6
8,demand_971,0,0.0,0.0,0.0,55490.7
9,demand_142,0,0.0,0.0,0.0,55470.9


In [140]:
combine(gdf_demand, [:LOL_MW => (x -> mean(x)) => :LOL_MW_mean, :LOL_MW => sum, :demand_MW => mean, :demand_MW => sum])

Row,demand,LOL_MW_mean,LOL_MW_sum,demand_MW_mean,demand_MW_sum
,Symbol,Float64,Float64,Float64,Float64
1,demand_831,0.0,0.0,2309.58,55429.9
2,demand_449,0.0,0.0,2308.03,55392.7
3,demand_48,0.0,0.0,2310.34,55448.1
4,demand_16,0.0,0.0,2310.6,55454.4
5,demand_165,0.0,0.0,2310.48,55451.4
6,demand_690,0.0,0.0,2307.89,55389.5
7,demand_940,0.0,0.0,2310.24,55445.6
8,demand_971,0.0,0.0,2312.11,55490.7
9,demand_142,0.0,0.0,2311.29,55470.9


In [141]:
# # combine(gdf_demand,
# #               AsTable([:LOL_MW, :demand_MW]) =>
# #                (x -> (a = std(x.LOL_MW) / std(x.demand_MW), b =  std(x.LOL_MW)))=> AsTable)

# combine(gdf_demand,
# AsTable([:LOL_MW, :demand_MW]) =>
# (x -> 
#     (LOL_hours=count(x.LOL_MW>0*x.demand_MW>0),
#     LOLP=count(x.LOL_MW.>0*x.demand_MW.>0)/count(x.demand_MW.>0),
#     LOL_MWh = sum(x.LOL_MW),
#     LOL_percentage = sum(x.LOL_MW)/sum(x.demand_MW),
#     Demand_MWh = sum(x.demand_MW),
#     )
# ) => AsTable)